# BirdWatch Timeseries for Location Clusters

## Author: Rosemarie Martienssen

This notebook is the forth in a series of (so far) four. In the first notebook, we cleaned and concatanated Datasets obtained from https://feederwatch.org/explore/raw-dataset-requests/ . The end result was one dataset containing Data ranging between the years 2011-2023. 
The second notebook featured an initial attempt at using timeseries to model bird sightings. The approach from notebook 2 will be mirrored here (with some enhancements) but we will also incorporate the results from Notebook 3, where we used Clustering to identify different locations. The goal is to have a model for each bird species and each location to predict the occurence at the species at the given location.

### Table of contents <a class = "anchor" id = "ToC"></a>
- [Part 0: Import required Python packages and import data](#Part-0)

- [Part 1: Prepare for time series](#Part-1)
 
- [Part 2:Fit models using Facebook Prophet](#Part-2)
    


### Part 0: Import required Python packages and import  data <a class = "anchor" id = "Part-0"></a>

In [1]:
#import packages

import pandas as pd
import numpy as np

# plotting
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objs as go
import matplotlib.pyplot as plt
import seaborn as sns

# stats
from statsmodels.api import tsa # time series analysis
import statsmodels.api as sm
from statsmodels.graphics.tsaplots import month_plot
from statsmodels.tsa.seasonal import seasonal_decompose

#FB prophet model
from prophet import Prophet

C:\Users\rmart\.conda\envs\timeseries\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
#Import the source data for this notebook

df_raw = pd.read_csv("../data/CA_BC_clustered_feederwatch_2011_2023.csv")

C:\Users\rmart\AppData\Local\Temp\ipykernel_6532\1433707354.py:3: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df_raw = pd.read_csv("../data/CA_BC_clustered_feederwatch_2011_2023.csv")


In [3]:
df_raw.shape

(516216, 22)

In [4]:
df_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 516216 entries, 0 to 516215
Data columns (total 22 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   LOC_ID                 516216 non-null  object 
 1   LATITUDE               516216 non-null  float64
 2   LONGITUDE              516216 non-null  float64
 3   SUBNATIONAL1_CODE      516216 non-null  object 
 4   SUB_ID                 516216 non-null  object 
 5   OBS_ID                 516216 non-null  object 
 6   Month                  516216 non-null  int64  
 7   Day                    516216 non-null  int64  
 8   Year                   516216 non-null  int64  
 9   PROJ_PERIOD_ID         516216 non-null  object 
 10  alt_full_spp_code      4943 non-null    object 
 11  HOW_MANY               516216 non-null  float64
 12  DAY1_AM                516216 non-null  int64  
 13  DAY1_PM                516216 non-null  int64  
 14  DAY2_AM                516216 non-nu

***
[Back to Table of Contents](#ToC) 
***

### Part 1: prepare for time series <a class = "anchor" id = "Part-1"></a>

**Remove columns**

For our time-series, we will not need the majority of columns currently in the dataframe. We can create a new one that just focuses on the data we need.

In [5]:
#Create a new dataframe and display the first rows

df = df_raw[['date', 'american_english_name', 'HOW_MANY', 'Cluster']]
df.head()

,date,american_english_name,HOW_MANY,Cluster
0,2011-02-07,White-breasted Nuthatch,2.0,4
1,2010-11-22,White-breasted Nuthatch,1.0,0
2,2010-12-02,White-breasted Nuthatch,1.0,4
3,2010-12-09,White-breasted Nuthatch,5.0,4
4,2011-03-06,White-breasted Nuthatch,2.0,5


The `Cluster` column is based on the Location clustering we performed in Notebook 03. We have a total number of 8 clusters in the dataset and will divide the current dataframe into 8 different dataframes to obtain models based on the location.

**Divide into Clusters**

In [6]:
#Create 8 different dataframes for the 8 clusters

for i in range(8):
    globals()[f"df_{i}"] = df[df['Cluster'] == i]


In [7]:
#Sanity Check

df_0.head(2)

,date,american_english_name,HOW_MANY,Cluster
1,2010-11-22,White-breasted Nuthatch,1.0,0
41,2012-12-09,White-breasted Nuthatch,2.0,0


In [8]:
#sanity Check

df_1.head(2)

,date,american_english_name,HOW_MANY,Cluster
16,2012-11-17,White-breasted Nuthatch,1.0,1
234,2011-12-31,White-breasted Nuthatch,1.0,1


In [9]:
#For each of the Clusters, show the bird species that occur most often in the dataframe

for i in range(8):
    print(i)
    display(globals()[f"df_{i}"]['american_english_name'].value_counts().head(5))

0


american_english_name
Dark-eyed Junco           15470
Black-capped Chickadee    15345
Song Sparrow              12511
Spotted Towhee            10996
Anna's Hummingbird        10222
Name: count, dtype: int64

1


american_english_name
Black-capped Chickadee    3994
Downy Woodpecker          2699
Northern Flicker          2550
Red-breasted Nuthatch     2479
Hairy Woodpecker          2132
Name: count, dtype: int64

2


...

american_english_name
Dark-eyed Junco              10512
Anna's Hummingbird            9556
Chestnut-backed Chickadee     9279
Spotted Towhee                9184
Northern Flicker              6615
Name: count, dtype: int64

6


american_english_name
Dark-eyed Junco              1561
Black-capped Chickadee       1202
Steller's Jay                1157
Chestnut-backed Chickadee     815
Pine Siskin                   716
Name: count, dtype: int64

7


american_english_name
Black-capped Chickadee    798
Blue Jay                  660
Downy Woodpecker          588
Hairy Woodpecker          569
Common Redpoll            517
Name: count, dtype: int64

These results show that there is in fact a difference in the locations because we see different bird patterns, even though there are also some similarities.

**Create Dummy Variables for the bird species**


In [10]:
#Create dummy variables for the column "american_english name" for each location cluster

for i in range(8):
    df = globals()[f"df_{i}"]
    globals()[f"df_dummy_{i}"] = pd.get_dummies(df, columns=['american_english_name'], prefix='no_of', dtype=int)

In [11]:
#For each of the location clusters, show the size of the dataframe and the first row

for i in range(8):
    print(i)
    display(globals()[f"df_dummy_{i}"].shape)
    display(globals()[f"df_dummy_{i}"].head(1))

0


(172551, 117)

,date,HOW_MANY,Cluster,no_of_Accipiter sp.,no_of_American Crow,no_of_American Goldfinch,no_of_American Kestrel,no_of_American Robin,no_of_American Tree Sparrow,no_of_American Wigeon,...,no_of_Western Meadowlark,no_of_White-breasted Nuthatch,no_of_White-crowned Sparrow,no_of_White-throated Sparrow,no_of_Winter Wren,no_of_Wood Duck,no_of_Yellow-rumped Warbler,no_of_Yellow-rumped Warbler (Audubon's),no_of_hawk sp.,no_of_rosy-finch sp.
1,2010-11-22,1.0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0


1


(29803, 88)

...

(14240, 77)

,date,HOW_MANY,Cluster,no_of_American Crow,no_of_American Goldfinch,no_of_American Kestrel,no_of_American Robin,no_of_American Tree Sparrow,no_of_Anna's Hummingbird,no_of_Bald Eagle,...,no_of_Song Sparrow,no_of_Spotted Towhee,no_of_Steller's Jay,no_of_Varied Thrush,no_of_White-breasted Nuthatch,no_of_White-crowned Sparrow,no_of_White-throated Sparrow,no_of_White-winged Crossbill,no_of_Winter Wren,no_of_Yellow-rumped Warbler
866,2018-12-02,1.0,6,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0


7


(6263, 51)

,date,HOW_MANY,Cluster,no_of_American Crow,no_of_American Robin,no_of_American Tree Sparrow,no_of_Bald Eagle,no_of_Black-billed Magpie,no_of_Black-capped Chickadee,no_of_Blue Jay,...,no_of_Red-winged Blackbird,no_of_Rock Pigeon (Feral Pigeon),no_of_Ruffed Grouse,no_of_Sharp-shinned Hawk,no_of_Snow Bunting,no_of_Steller's Jay,no_of_White-breasted Nuthatch,no_of_White-crowned Sparrow,no_of_White-throated Sparrow,no_of_White-winged Crossbill
5,2011-11-19,1.0,7,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0


Because different bird species appear in different clusters, the number of columns varies across the different dataframes.

The data now represents whether or not a certain species was seen in a day. We will combine the `HOW_MANY` column with the dummy columns `no_of___`

In [12]:
#Multiply the dummy variables by the "How many" column

for i in range(8):
    df = globals()[f"df_dummy_{i}"]
    bird_species = df.columns[3:] # All the columns containing bird species
    for col in bird_species:
       globals()[f"df_dummy_{i}"][col] *= df['HOW_MANY']

In [13]:
#Drop the "How Many" column and the "Cluster from each dataframe

for i in range(8):
    df = globals()[f"df_dummy_{i}"]
    globals()[f"df_dummy_{i}"].drop(columns=['HOW_MANY'], inplace=True)
    globals()[f"df_dummy_{i}"].drop(columns=['Cluster'], inplace=True)


**Fill in missing dates**

We will group the data by the date columns to have one row of data for each day. Furthermore, we will have to fill in any missing dates with zeros in order to have continuos data.

In [14]:
# Group each dataframe by the date column and set it as the new index

for i in range(8):
    df = globals()[f"df_dummy_{i}"]
    globals()[f"df_dummy_{i}"]= df.groupby(['date']).sum()
    globals()[f"df_dummy_{i}"].reset_index().set_index('date')

In [16]:
#For each of the dataframes, save the first and last name in a list to compare timeframes

first_day = []
last_day = []
for i in range(8):
    df = globals()[f"df_dummy_{i}"]
    first = df.index.min()
    last = df.index.max()
    first_day.append(first)
    last_day.append(last)

print(first_day)
print(last_day)
    

['2010-11-01', '2010-11-13', '2010-11-13', '2010-11-13', '2010-11-13', '2010-11-13', '2010-11-13', '2010-11-13']
['2023-04-29', '2023-04-29', '2023-04-29', '2023-04-29', '2023-04-29', '2023-04-29', '2023-04-29', '2023-04-28']


The date ranges are overall very similar. We will work with these ranges and not adjust them additionally by making them all the exact same. Instead, we will just identify missing values for each dataframe within its individual range and fill these dates with zeros. Note that the survey collecting observations is only active in the Winter months, which is why a lot of missing dates during the summer are expected.

In [29]:
# Add the missing dates 

for i in range(8):
    full_range = pd.date_range(start=first_day[i], end=last_day[i], freq="D")
    df = globals()[f"df_dummy_{i}"]
    globals()[f"df_dummy_{i}"] = df.reindex(full_range)
    full_range.difference(df.index)



In [37]:
# Fill in missing values with zeros

for i in range(8):
    df = globals()[f"df_dummy_{i}"]
    globals()[f"df_clean_{i}"] = df.fillna(0)


In [40]:
#Check for any missin values

for i in range(8):
    df = globals()[f"df_clean_{i}"]
    print(df_clean_0.isna().sum().sum())

0
0
0
0
0
0
0
0


#### Convert to monthly data

The data is now ready for modelling. We are not scaling the data right now.

***
[Back to Table of Contents](#ToC) 
***

### Part 2: Fit Models using Facebook Prophet <a class = "anchor" id = "Part-2"></a>